In [17]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv("https://raw.githubusercontent.com/Adelaaas/Data_science_advanced_2022/main/%D0%9D%D0%A2%D0%9E%20%D0%B8%D0%BD%D0%B6%20%D1%82%D1%83%D1%80/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/Adelaaas/Data_science_advanced_2022/main/%D0%9D%D0%A2%D0%9E%20%D0%B8%D0%BD%D0%B6%20%D1%82%D1%83%D1%80/predict.csv")

train

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,6.901420,24.569658,0.862829,0.737416,G5V,20.449126,1
1,4.334196,2.198947,0.593309,1.153210,G2Ib,13.043324,0
2,7.873019,11.750465,0.616568,0.589166,F5V,17.736492,1
3,8.110014,2.143815,0.848508,1.322107,K2III,15.592469,0
4,8.025193,4.165622,0.899296,1.141646,K5III,16.379680,0
...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,A0II,12.542120,0
31637,8.078011,3.002784,0.689891,0.971966,G8III,15.379710,0
31638,7.065964,7.820498,0.885794,0.435381,F3IV/V,16.737442,1
31639,8.685452,2.643033,1.064647,0.988600,K0III,14.859882,0


In [18]:
test

,Vmag,Plx,e_Plx,B-V,SpType,Amag
0,4.039519,13.596504,0.845101,-0.097823,F5V+...,15.774122
1,6.655719,4.010332,1.050017,1.386670,K5III,15.344976
2,6.405060,0.348855,0.961598,0.951503,B0IVpe,12.236103
3,9.113068,2.954461,1.251422,1.002819,K1III,15.560838
4,6.107686,3.144180,0.475795,0.493345,G8II-IIIvar,12.771981
...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,M3III,16.438014
7907,6.255358,3.034436,0.449386,0.966053,K0III,13.898403
7908,8.643598,10.932145,0.947897,0.504383,G0V,18.980729
7909,9.400810,6.279361,1.332124,0.351073,F2V,16.872491


In [19]:
train.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31641 entries, 0 to 31640
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Vmag         31641 non-null  float64
 1   Plx          31641 non-null  float64
 2   e_Plx        31641 non-null  float64
 3   B-V          31641 non-null  float64
 4   SpType       31641 non-null  object 
 5   Amag         31641 non-null  float64
 6   TargetClass  31641 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 3.3 MB


In [20]:
test.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7911 entries, 0 to 7910
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Vmag    7911 non-null   float64
 1   Plx     7911 non-null   float64
 2   e_Plx   7911 non-null   float64
 3   B-V     7911 non-null   float64
 4   SpType  7911 non-null   object 
 5   Amag    7911 non-null   float64
dtypes: float64(5), object(1)
memory usage: 787.2 KB


In [21]:
train["Class"] = 0
train["Luminosity"] = 0

for index, row in train.iterrows():
    train.loc[index, "Class"] = re.findall(r'([A-Z])', train.loc[index, "SpType"])[0]
    try:
        m = re.findall(r'([0-9])', train.loc[index, "SpType"])[0]
        n = re.findall(r'.*[0-9]([IV]*)', train.loc[index, "SpType"])[-1]
        if n not in ["I", "II", "III", "IV", "V", "VI", "VII"]:
            train.loc[index, "Luminosity"] = "IV"
        else:
            train.loc[index, "Luminosity"] = n
        train.loc[index, "SubClass"] = m
    except IndexError:
        train.loc[index, "Luminosity"] = "IV"
        train.loc[index, "SubClass"] = 4

sp_type = train.drop("SpType", axis=1, inplace=True)

train

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,Class,Luminosity,SubClass
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,G,V,5
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,G,I,2
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,F,V,5
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,K,III,2
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,K,III,5
...,...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,12.542120,0,A,II,0
31637,8.078011,3.002784,0.689891,0.971966,15.379710,0,G,III,8
31638,7.065964,7.820498,0.885794,0.435381,16.737442,1,F,IV,3
31639,8.685452,2.643033,1.064647,0.988600,14.859882,0,K,III,0


In [22]:
le1 = LabelEncoder()
le2 = LabelEncoder()

label_1 = le1.fit_transform(train["Class"])
label_2 = le2.fit_transform(train["Luminosity"])

train.drop("Class", axis=1, inplace=True)
train.drop("Luminosity", axis=1, inplace=True)

train["Class"] = label_1
train["Luminosity"] = label_2

train

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,SubClass,Class,Luminosity
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,5,4,4
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,2,4,0
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,5,3,4
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,2,5,2
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,5,5,2
...,...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,12.542120,0,0,0,1
31637,8.078011,3.002784,0.689891,0.971966,15.379710,0,8,4,2
31638,7.065964,7.820498,0.885794,0.435381,16.737442,1,3,3,3
31639,8.685452,2.643033,1.064647,0.988600,14.859882,0,0,5,2


In [23]:
%%time

x = train.drop(columns=["TargetClass"])
y = train["TargetClass"]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.3)

rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
rfc.fit(x_train, y_train)

CPU times: user 4.65 s, sys: 21.6 ms, total: 4.67 s
Wall time: 2.58 s


RandomForestClassifier(n_jobs=-1, random_state=42)

In [24]:
predictions = rfc.predict(x_test)
roc_auc_score(y_test, predictions)

0.9593466452071661

In [25]:
test["Class"] = 0
test["Luminosity"] = 0

for index, row in test.iterrows():
    test.loc[index, "Class"] = re.findall(r'([A-Z])', test.loc[index, "SpType"])[0]
    try:
        m = re.findall(r'([0-9])', test.loc[index, "SpType"])[0]
        n = re.findall(r'.*[0-9]([IV]*)', test.loc[index, "SpType"])[-1]
        if n not in ["I", "II", "III", "IV", "V", "VI", "VII"]:
            test.loc[index, "Luminosity"] = "IV"
        else:
            test.loc[index, "Luminosity"] = n
        test.loc[index, "SubClass"] = m
    except IndexError:
        test.loc[index, "Luminosity"] = "IV"
        test.loc[index, "SubClass"] = 4

sp_type = test.drop("SpType", axis=1, inplace=True)

test

,Vmag,Plx,e_Plx,B-V,Amag,Class,Luminosity,SubClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,F,V,5
1,6.655719,4.010332,1.050017,1.386670,15.344976,K,III,5
2,6.405060,0.348855,0.961598,0.951503,12.236103,B,IV,0
3,9.113068,2.954461,1.251422,1.002819,15.560838,K,III,1
4,6.107686,3.144180,0.475795,0.493345,12.771981,G,II,8
...,...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,16.438014,M,III,3
7907,6.255358,3.034436,0.449386,0.966053,13.898403,K,III,0
7908,8.643598,10.932145,0.947897,0.504383,18.980729,G,V,0
7909,9.400810,6.279361,1.332124,0.351073,16.872491,F,V,2


In [26]:
le1 = LabelEncoder()
le2 = LabelEncoder()

label_1 = le1.fit_transform(test["Class"])
label_2 = le2.fit_transform(test["Luminosity"])

test.drop("Class", axis=1, inplace=True)
test.drop("Luminosity", axis=1, inplace=True)

test["Class"] = label_1
test["Luminosity"] = label_2

test

,Vmag,Plx,e_Plx,B-V,Amag,SubClass,Class,Luminosity
0,4.039519,13.596504,0.845101,-0.097823,15.774122,5,3,4
1,6.655719,4.010332,1.050017,1.386670,15.344976,5,5,2
2,6.405060,0.348855,0.961598,0.951503,12.236103,0,1,3
3,9.113068,2.954461,1.251422,1.002819,15.560838,1,5,2
4,6.107686,3.144180,0.475795,0.493345,12.771981,8,4,1
...,...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,16.438014,3,6,2
7907,6.255358,3.034436,0.449386,0.966053,13.898403,0,5,2
7908,8.643598,10.932145,0.947897,0.504383,18.980729,0,4,4
7909,9.400810,6.279361,1.332124,0.351073,16.872491,2,3,4


In [28]:
prediction = pd.Series(rfc.predict(test))
prediction

0       1
1       0
2       1
3       0
4       0
       ..
7906    0
7907    0
7908    1
7909    1
7910    0
Length: 7911, dtype: int64

In [29]:
prediction.to_csv("Predictions.csv")
prediction_file = pd.read_csv("/content/Predictions.csv")
prediction_file

,Unnamed: 0,0
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
7906,7906,0
7907,7907,0
7908,7908,1
7909,7909,1


In [30]:
prediction_file.rename(columns={"Unnamed: 0": "Index", "0": "TargetClass"}, inplace=True)
prediction_file

,Index,TargetClass
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
7906,7906,0
7907,7907,0
7908,7908,1
7909,7909,1


In [31]:
test = test.join(prediction_file)
test

,Vmag,Plx,e_Plx,B-V,Amag,SubClass,Class,Luminosity,Index,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,15.774122,5,3,4,0,1
1,6.655719,4.010332,1.050017,1.386670,15.344976,5,5,2,1,0
2,6.405060,0.348855,0.961598,0.951503,12.236103,0,1,3,2,1
3,9.113068,2.954461,1.251422,1.002819,15.560838,1,5,2,3,0
4,6.107686,3.144180,0.475795,0.493345,12.771981,8,4,1,4,0
...,...,...,...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,16.438014,3,6,2,7906,0
7907,6.255358,3.034436,0.449386,0.966053,13.898403,0,5,2,7907,0
7908,8.643598,10.932145,0.947897,0.504383,18.980729,0,4,4,7908,1
7909,9.400810,6.279361,1.332124,0.351073,16.872491,2,3,4,7909,1


In [32]:
final_predictions = test.drop(["Index"], axis=1)
final_predictions = final_predictions.reindex(columns=["Vmag", "Plx", "e_Plx", "B-V", "SpType", "Amag", "TargetClass"])
final_predictions

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,4.039519,13.596504,0.845101,-0.097823,NaN,15.774122,1
1,6.655719,4.010332,1.050017,1.386670,NaN,15.344976,0
2,6.405060,0.348855,0.961598,0.951503,NaN,12.236103,1
3,9.113068,2.954461,1.251422,1.002819,NaN,15.560838,0
4,6.107686,3.144180,0.475795,0.493345,NaN,12.771981,0
...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,NaN,16.438014,0
7907,6.255358,3.034436,0.449386,0.966053,NaN,13.898403,0
7908,8.643598,10.932145,0.947897,0.504383,NaN,18.980729,1
7909,9.400810,6.279361,1.332124,0.351073,NaN,16.872491,1


In [33]:
from google.colab import files
uploaded = files.upload()

final_predictions.to_csv("Final_Predictions.csv", index=False)
files.download("Final_Predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>